# Overview

I found a fun short project recently, set by the online fashion company Zalando to people applying for their Data Science positions. The goal of the project is to locate the whereabouts of a new analyst that they wish to recruit, given some limited information about her. My code below suggests three high probability locations:

1. a power plant called Heizkraftwerk Klingenberg at Lat/Lon coordinates (52.4911, 13.4948).
2. the Zalando Content Creation office at (52.5094, 13.4367).
3. Technische Universtität Berlin's Produktionstechnisches Zentrum at (52.5247, 13.3222).

In this notebook I’ll go through my implementation, both the python code and the small amount of maths needed. In the course of doing research for the project I found a couple of nice alternative solutions: see the blogs of [Aonghus Lawlor](http://aonghuslawlor.com/blog/zalando-challenge.html) (no relation) and [Yaser Martinez](http://yasermartinez.com/zalando/report.html). I have to admit both of them have produced a much more visually appealing result than my code can manage currently, however we do get similar results for the analyst location. I had a lot of issues with the map tiling needed to generate a result similar to theirs, when I learn more about them I hope to come back to this mini project and fix it.

The problem itself involves modelling three probability distributions over a Cartesian plane that represents Berlin. By combining these distributions and finding the maxima it’s possible to figure out the most likely location(s). The solution below is laid out as follows:
1. For self-containment I’ll first recap the challenge brief as it is set by Zalando.
2. Probability Distributions for the three data sources.
3. Shortest Euclidean distance from a point to a line segment
4. Coordinate Transform. Her coordinates are given in terms of latitude and longitude but they’ve been kind enough to give us a decent coordinate transform that takes us to a Cartesian plane (i.e. flat instead of spherical).
5. Python implementation for high probability locations
6. Heatmap plot
7. Where is the analyst?

## 1 - Challenge Brief

The Zalando Data Intelligence Team is searching for a new top analyst. We already know of an excellent candidate with top analytical and programming skills. Unfortunately, we don’t know her exact whereabouts but we only have some vague information where she might be. Can you tell us where to best send our recruiters and plot an easy to read map of your solution for them?  This is what we could extract from independent sources:

* The candidate is likely to be close to the river Spree. The probability at any point is given by a Gaussian function of its shortest distance to the river. The function peaks at zero and has 95\% of its total integral within +-2730m.
* A probability distribution centred around the Brandenburg Gate also informs us of the candidate’s location. The distribution’s radial profile is log-normal with a mean of 4700m and a mode of 3877m in every direction.
* A satellite offers further information: with 95% probability she is located within 2400 m distance of the satellite’s path (assuming a normal probability distribution).

In [16]:
import numpy as np
from scipy.stats import lognorm, norm
import scipy.interpolate
from scipy import optimize

## 2 - Probability Distributions

Given the information in the challenge brief, three probability distributions ($P_\text{Spree}$, $P_\text{Sat.}$ and $P_\text{BBG}$) are now defined which will later be combined to yield a total probability distribution ($P$). We assume that all three sources of information are equally reliable so that $P = \frac{P_\text{Spree} + P_\text{Sat.} + P_\text{BBG}}{3}$, however they could be weighted later on if desired. 

### Spree distribution parameters

We need to find the parameters of the Gaussian function so we can plot it - these are the standard deviation and the mean, however we are told the mean is zero. We can figure out the standard deviation from the error function and the knowledge that 95\% of the total integral is within a distance 2.73km using the formula 
$$ F(\mu + n \sigma) - F(\mu - n \sigma) = \text{erf} \frac{n}{\sqrt{2}}  $$
where F is the cumulative distribution function of the normal distribution with mean $\mu = 0$, standard deviation $\sigma$ and $n$ varies depending on the confidence interval required, e.g. for 2 standard deviations $n = 2$. As $F(\mu + n \sigma) - F(\mu - n \sigma) = 0.95$ we rearrange the above to find
$$ n = \sqrt{2} \text{erf}^{-1} (0.95) = 1.96 .$$
We know that
$$ n \sigma = 2.730 $$
so
$$ \sigma = \frac{2.730}{1.95996} = 1.39289. $$
 
### Satellite distribution parameters
Similar to the above, we know $n \sigma = 2.4$km so by the same method
$$  \sigma = \frac{2.4}{\text{erf}^{-1}(0.95)} = 1.22451.$$
 


In [17]:
def p_sat(x,y,
            start_x = 52.437385, start_y = 13.553989,
            end_x =52.590117, end_y = 13.39915):

    """
    normal distribution along satellite path, returns value of distribution at
    point (x,y) in Cartesian coordinates given satellite path between 
    (start_x,start_y) and (end_x,end_y) in Lat/Lon coordinate system.
    """
    sgm =  1.22451    # SD of norm for this data source. see pdf doc
    # Transform start and end points of satellite path to cartesian
    start_x , start_y = t(start_x,start_y)  
    end_x, end_y = t(end_x,end_y)
    #calculate distance d from (x,y) to path
    d = d_to_seg(x,y,np.array([start_x, start_y]), np.array([end_x, end_y]))    
    norm_sat = norm(0, sgm) # Create normal distribution
    return norm_sat.pdf(d)  # Return value of point (x,y) at distance d

def p_spree(x,y):

    """
    Normal distribution along the Spree.
    Similar to p_sat except we work with several joined line segments
    """ 
    # Load coords from external data file
    coords = np.loadtxt('spree_coords.dat',delimiter=',',dtype=float)  
    #Produce line segment representation of the spree:
    segments = [np.array(
                    [np.array(t(coords[q][0],coords[q][1])),
                    np.array(t(coords[q+1][0],coords[q+1][1]))]
                ) for q in range(len(coords)-1)]    
    sgm = 1.39289   # SD of norm, see PDF document

    # function measures (x,y) to all Spree line segments,
    # returns shortest distance
    def spree_f(xs,ys):
        return min([d_to_seg(xs,ys,s[0],s[1]) for s in segments ])          

    v_spree_f = np.vectorize(spree_f)
    d = v_spree_f(x,y)
    norm_spree = norm(0,sgm)
    return norm_spree.pdf(d)    


### Brandenburg Gate distribution parameters
Log-normal distributions are parametrised by $\mu$ and $\sigma$, called the location and scale parameters respectively, which we will calculate here for the Brandenburg gate distribution. We are told the mean of this distribution is $m = 4.700$km and the mode $a=3.877$km in all directions. These quantities can be expressed by the formulae:
$$ m = e^{\mu + \frac{\sigma^2}{2}} $$
$$ a = e^{\mu - \sigma^2}. $$
This forms a system of 2 solved equations with 2 unknown parameters ($\mu$ and $\sigma$) and can be solved relatively easily by hand
$$ \mu = \log(a) +  \sigma^2 $$
which is substituted into
\begin{equation*}
\begin{split}
\sigma^2 &= 2 ( \ln{4.7} - \mu ) \\
 &=  2 ( \ln{4.7} - \ln{3.877} ) - 2 \sigma^2   \\
\therefore \sigma &= \sqrt{\frac{2}{3} \ln{\frac{4.7}{3.877}}} \\
&= 0.3582
\end{split}
\end{equation*}
so $ \mu = 1.4834 $.

In [18]:
def p_bbg(x,y,
        center_x = 52.516288,
        center_y = 13.377689 ):

    """
    Log-normal distribution around the Brandenburg Gate (BBG)
    Returns the probability of the x,y input being the analyst
    location according to the BBG source.
    """
    x0 , y0 = t(center_x, center_y) # Transform lat,lon of the BBG to cartesian
    # parameters for the distribution, see accompanying pdf document
    mu = 1.4834  ; sigma = 0.3582  
    ln_bb = lognorm(s=sigma, scale = np.exp(mu)) # Creates log-norm distribution
    return ln_bb.pdf(euclid(x,y,x0,y0)) # Return distribution val at (x,y)


## 3 - Shortest Euclidean distance from a point to a line segment

<img src="https://github.com/jameslawlor/zalando_datascience_teaser/blob/master/vectors.png?raw=true">

Here we will show how to calculate the nearest Euclidean distance between a point $p$ and a line segment passing through points $p_0$ and $p_1$. Consider the point $p$ shown in the figure above which is joined to the segment by the vectors $\vec{w}_0$ and $\vec{w}_1$. If 
$$\vec{w}_0 \cdot \vec{v} < 0$$
then it follows the closest point to $p$ on the segment must be $p_0$. Similarly if 
$$\vec{w}_1 \cdot \vec{v} > 0$$
then the closest point to $p$ is $p_1$. If $p$ lies somewhere such that neither of these conditions is fulfilled then it must lie perpendicular to a point lying on the segment itself.By parameterising the line segment such that $$\vec{v}(b) = p_1 + b ( p_0 - p_1)$$ 
where 
$$b = \frac{\vec{w}_0 \cdot \vec{v}}{\vec{v} \cdot \vec{v}}$$
it is the possible to find the point on the line closest to $p$. Note the efficiency of this method - only the dot products $\vec{w}_0 \cdot \vec{v}$ and $\vec{v} \cdot \vec{v}$ are required to find the point on the line segment. When this point is known the Euclidean distance to point $p$ can be calculated easily.

In [19]:
def euclid(x,y,x0=0,y0=0):
    """
    Euclidean distance between cartesian points (x,y) and (x0,y0)
    """
    return np.sqrt((x-x0)**2 + (y-y0)**2)

def d_to_seg(xx,yy,p0,p1):

    """
    Calculates Euclidean distance between a point (xx,yy)
    and a line segment with start & end points p0, p1
    for details of how this works please see the attached PDF
    """
    def g(x0,y0):   
        v = p1 - p0                     # Vector of line
        w = np.array([x0,y0]) - p0      # Vector from point to line start point
        
        v1 = np.dot(w,v)
        v2 = np.dot(v,v)
        # Point lies to left of line, closest point is p0
        if v1 <= 0.0:    return euclid(x0,y0,p0[0],p0[1])  
        # Point lies to right, closest point is p1.
        if v2 <= v1:     return euclid(x0,y0,p1[0],p1[1])   

        b = v1/v2             # If neither if statement is satisfied, closest
        pb = p0 + b*v         #    segment point must lie between p0 and p1
        return euclid(x0,y0,pb[0],pb[1])

    # As we work with meshgrid, this allows func to operate
    # on all XY values simultaneously
    vf = np.vectorize(g)     
    return vf(xx,yy)

## 4 - Coordinate Transform 

First a coordinate transform function t is defined which can convert between Latitude/Longitude and Cartesian (or vice versa by setting inv=True, which will be useful later when we want to map our coordinates back into something we can find on Google Maps). This transform was provided to us in the problem brief on the Zalando website and is quite accurate within the Berlin area. 

In [20]:
def t(xx,yy,inv=False):
    """
    Coordinate transform between Lat/Lon <--> Cartesian
    accurate for the Berlin Area.
    """
    SW_lat = 52.464011 #(Latitude)
    SW_lon = 13.274099 #(Longitude)

    if inv:     # Lat/Lon -> Cartesian
        lon = SW_lon + (xx/111.323)/(np.cos(SW_lat*np.pi/180.0))
        lat = (yy/111.323) + SW_lat
        return lat, lon
    else:       # Cartesian -> Lat/Lon
        x = (yy - SW_lon) * np.cos(SW_lat * np.pi / 180.0) * 111.323
        y = (xx - SW_lat) * 111.323
        return x, y

## 5 - Implementation

Given the above functions we can now try and find the analyst. A XY grid is made using numpy's meshgrid and the probability measured in the Z-direction. The resulting surface is then optimised via a Brute Force method (used as the problem is fairly small scale) and the coordinate of maximum probability is found.

As will be shown in the next section, there are actually a handful of possible locations and these become evident when the surface is plotted. By taking a small slice of the XY grid around these points the same algorithm can be used to find these exact locations also, and I've commented out the three main ones below. Currently this will return the global maximum probability at (52.491055477900282, 13.494788009568131) where the inverse coordinate transform was used to get this in Lat/Lon.

In [21]:
if __name__ == "__main__":
    
    delta = 5.0e-1  # Resolution of XY grid
   
    # Build grid of XY cartesian coordinates 
    X, Y = np.meshgrid(np.arange(0.0, 20.01, delta),
             np.arange(-5.0, 15.01, delta))
    # Find the 3 distributions
    Z_spree = p_spree(X,Y)
    Z_sat = p_sat(X,Y)
    Z_bbg = p_bbg(X,Y)
    # Combined probability P 
    P = (Z_spree + Z_sat + Z_bbg )/3.0

    #################################
    # Optimisation
    #################################
    #################################
    def prob(z):
        X,Y = z
        Z_spree = p_spree(X,Y)
        Z_sat = p_sat(X,Y)
        Z_bbg = p_bbg(X,Y)
        return 1.0 - (Z_spree + Z_sat + Z_bbg)/3.0
    # Heizkraftwerk
    #rranges = (slice(-0.0, 20.0, delta), slice(-5.0, 15.0, delta)) 
    # TU Berlin
    #rranges = (slice(-0.0, 10.0, delta), slice(-0.0, 10.0, delta)) 
    # Zalando CC office
    #rranges = (slice(-5.0, 14.0, delta), slice(-5.0, 15.0, delta))
    resbrute = optimize.brute(prob, rranges, args=(), full_output=True,
                                   finish=optimize.fmin)
    print 'Global Max:'
    print 'P = ', 1-resbrute[1]
    #print 'Cartesian Coords = ', resbrute[0][0], resbrute[0][1]
    # 
    print 'Lat/Lon = ', t(resbrute[0][0], resbrute[0][1],True)


Global Max:
P =  0.207081084144
Lat/Lon =  (52.491055477900282, 13.494788009568131)


## 6 - Heatmap Plot

The surface can be interpolated and plotted (code below)

<img src="https://raw.githubusercontent.com/jameslawlor/zalando_datascience_teaser/master/heatmap.png">
Heat map of $P(x,y)$ in Cartesian coordinates. 
 The data source coordinates/paths of the Brandenburg Gate, satellite and Spree are shown by the red circle, red line and green line respectively.
 Local maxima in the probability of the analyst location are indicated by the blue triangles, corresponding to (L-R): TU Berlin, Zalando Content Creation and the power plant.

In [22]:
import matplotlib.pyplot as plt
plt.figure()

rbf = scipy.interpolate.Rbf(X, Y, P, function='linear')
z = P
zi = rbf(X, Y)

plt.imshow(zi, vmin=z.min(), vmax=z.max(), origin='lower',
           extent=[X.min(), X.max(), Y.min(), Y.max()],
             cmap=plt.get_cmap('hot'))
plt.grid(which='major', axis='both', linestyle='-', color='k')

# Brandenburg Gate point
bbg_x, bbg_y = t(52.516288,13.377689)
plt.plot(bbg_x,bbg_y,'o',color='r',markersize=15)

# Satellite path
sat_x_start, sat_y_start = t( 52.437385, 13.553989)
sat_x_end, sat_y_end= t(52.590117, 13.39915)
plt.plot([sat_x_start,sat_x_end], [sat_y_start, sat_y_end],
             color='r', linestyle='-', linewidth=5)

#Spree path
coords = np.loadtxt('spree_coords.dat',delimiter=',',dtype=float) 
segments = [np.array([np.array(t(coords[q][0],coords[q][1])),
                np.array(t(coords[q+1][0],coords[q+1][1]))])
                 for q in range(len(coords)-1)]    
for s in segments:
    plt.plot( [s[0][0],s[1][0]],[s[0][1],s[1][1]] ,
            color = 'g', linestyle='-', linewidth=5    )

# Points of Interest
x, y = t(52.4911, 13.4948)  # Thermal plant
plt.plot(x,y,'v',color='b',markersize=15)
x, y = t(52.5094, 13.4367)  # TU Berlin
plt.plot(x,y,'v',color='b',markersize=15)#
x, y = t(52.5247, 13.3222)  # Zalando
plt.plot(x,y,'v',color='b',markersize=15)
plt.xlabel('X (km)')
plt.ylabel('Y (km)')
cb = plt.colorbar()
cb.set_label("Probability")
plt.show()


## 7 - Where is the Analyst?

A heat map of $P$ is shown in the above figure where we identify three distinct locations with high probabilities using the brute force optimisation algorithm.

  
* Highest probability ($p=0.207$) at coordinates (52.4911, 13.4948) at Heizkraftwerk Klingenberg, a power plant.
* Second highest probability ($p=0.194$) at (52.5094, 13.4367), the Zalando Content Creation office.
* Third highest ($p=0.185$) at (52.5247, 13.3222) corresponding to Technische Universtität Berlin: Produktionstechnisches Zentrum.

Intuition suggests that the third option is the most 'sensible' place to find the analyst although it is not the most probable according to the calculation. The power plant would be a surprising place to find such a skilled analyst and presumably if they were at the Zalando office their whereabouts would be known to the team already (or quickly checked). 

This could be explained by the fact we initially assumed all three data sources were equally reliable. If we were to have additional information on the accuracy of these sources it would be possible to weight the separate probability distributions accordingly, leading to a different total probability distribution $P$. For example, it can be immediately inferred from the figure that removing the satellite source would make the University and Zalando locations much more probable, as well as effectively removing the power plant as a point of interest.
 
Using the figure again we can also suggest that if the analyst is not at the three locations above, there exists a high probability path (an arête-like feature) between the power plant and the Zalando office along the North bank of the Spree. Potential locations along this path include a large Mercedes-Benz office, several nightclubs and a sports centre.
